In [13]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [14]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [15]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [16]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
25,model_12_9_20,0.996234,0.997362,0.991983,0.995788,0.003766,0.002528,0.003577,0.003022,"Hidden Size=[30], regularizer=0.1, learning_ra..."
26,model_12_9_21,0.996234,0.997362,0.991983,0.995788,0.003766,0.002528,0.003577,0.003022,"Hidden Size=[30], regularizer=0.1, learning_ra..."
27,model_12_9_10,0.996234,0.997362,0.991983,0.995788,0.003766,0.002528,0.003577,0.003022,"Hidden Size=[30], regularizer=0.1, learning_ra..."
28,model_12_9_24,0.996234,0.997362,0.991983,0.995788,0.003766,0.002528,0.003577,0.003022,"Hidden Size=[30], regularizer=0.1, learning_ra..."
29,model_12_9_19,0.996234,0.997362,0.991983,0.995788,0.003766,0.002528,0.003577,0.003022,"Hidden Size=[30], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
349,model_7_9_1,0.970317,0.985314,0.998339,0.992732,0.029683,0.012739,0.002113,0.007738,"Hidden Size=[20], regularizer=0.01, learning_r..."
358,model_6_9_7,0.969787,0.946221,0.938093,0.944364,0.030213,0.074821,0.030018,0.053737,"Hidden Size=[15], regularizer=0.1, learning_ra..."
375,model_6_9_6,0.968832,0.954189,0.945609,0.952216,0.031168,0.063736,0.026374,0.046154,"Hidden Size=[15], regularizer=0.1, learning_ra..."
398,model_14_8_3,0.967969,0.992457,0.993487,0.992893,0.032031,0.010380,0.007342,0.008950,"Hidden Size=[35], regularizer=0.1, learning_ra..."
